# Async LLM, LLMChain, Agent

Many operations in LangChain are i/o bound. Running concurrently offers significant advantages.

## Async LLMChain

In [3]:
import asyncio
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def generate_serially():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    for _ in range(5):
        resp = chain.run(product="toothpaste")
        print(resp)


async def async_generate(chain):
    resp = await chain.arun(product="toothpaste")
    print(resp)


async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tasks = [async_generate(chain) for _ in range(5)]
    await asyncio.gather(*tasks)


import time

s = time.perf_counter()
await generate_concurrently()
elapsed = time.perf_counter() - s
print(f"Concurrent executed in {elapsed:0.2f} seconds.")

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print(f"Serial executed in {elapsed:0.2f} seconds.")



MintySmile


SparklySmilesCo.


BrightSmile Toothpaste Company


BriteBeam Dental Care


Brushing Bright Inc.
Concurrent executed in 1.53 seconds.


SmileMakers Toothpaste


SparkleSmile Toothpaste.


ShineBright Toothpaste Co.


Smile Fresh Toothpaste.


BrightBrush Toothpaste
Serial executed in 4.93 seconds.


## Async Agent (with Tracing)

In [1]:
import asyncio
from langchain.agents import initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.tracers import LangChainTracer
from aiohttp import ClientSession

QUESTION = "Who won the US Open men's final in 2019? What is his age raised to the 0.334 power?"


def generate_serially():
    for _ in range(4):
        tracer = LangChainTracer()
        tracer.load_default_session()
        manager = CallbackManager([StdOutCallbackHandler(), tracer])
        llm = OpenAI(temperature=0, callback_manager=manager)
        tools = load_tools(["llm-math", "serpapi"], llm=llm)
        agent = initialize_agent(
            tools, llm, agent="zero-shot-react-description", verbose=True, callback_manager=manager
        )
        agent.run(QUESTION)


async def generate_concurrently():
    agents = []
    # To make async requests in Tools more efficient, you can pass in your own aiohttp.ClientSession, 
    # but you must manually close the client session at the end of your program/event loop
    aiosession = ClientSession()
    for color in ["blue", "green", "red", "pink"]:
        tracer = LangChainTracer()
        tracer.load_default_session()
        manager = CallbackManager([StdOutCallbackHandler(color=color), tracer])
        llm = OpenAI(temperature=0, callback_manager=manager)
        async_tools = load_tools(["llm-math", "serpapi"], llm=llm, aiosession=aiosession)
        agents.append(
            initialize_agent(async_tools, llm, agent="zero-shot-react-description", verbose=True, callback_manager=manager)
        )
    tasks = [async_agent.arun(QUESTION) for async_agent in agents]
    await asyncio.gather(*tasks)
    await aiosession.close()


import time

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print(f"Serial executed in {elapsed:0.2f} seconds.")

s = time.perf_counter()
await generate_concurrently()
elapsed = time.perf_counter() - s
print(f"Concurrent executed in {elapsed:0.2f} seconds.")



> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's final in 2019 and then calculate his age raised to the 0.334 power.
Action: Search
Action Input: "US Open men's final 2019 winner"
Observation: Rafael Nadal
Thought: I need to find out Rafael Nadal's age
Action: Search
Action Input: "Rafael Nadal age"
Observation: 36 years
Thought: I need to calculate 36 raised to the 0.334 power
Action: Calculator
Action Input: 36^0.334
Observation: Answer: 3.3098250249682484

Thought: I now know the final answer
Final Answer: Rafael Nadal, 36 years old, won the US Open men's final in 2019 and his age raised to the 0.334 power is 3.3098250249682484.

> Finished chain.


> Entering new AgentExecutor chain...
 I need to find out who won the US Open men's final in 2019 and then calculate his age raised to the 0.334 power.
Action: Search
Action Input: "US Open men's final 2019 winner"
Observation: Rafael Nadal
Thought: I need to find out Rafael Nadal's age
Action: Search